In [9]:
import torch
import torch.nn as nn
import onnx
import tensorflow as tf
import nobuco
from nobuco import ChannelOrder

In [10]:
class SimpleNet(torch.nn.Module):
    def __init__(self, input_size=6, output_size=4):
        super(SimpleNet, self).__init__()
        h_size = 32
        self.fc1 = nn.Linear(input_size, h_size)
        # self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(h_size, h_size)
        self.fc4 = nn.Linear(h_size, h_size)
        self.fc5 = nn.Linear(h_size, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        # x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.relu(self.fc4(x))
        x = self.fc5(x)
        return x

In [11]:
# Load the torch model
torch_model_path = 'models/TA_optimization_NN_cpu.pt'
torch_model = SimpleNet()
torch_model.load_state_dict(torch.load(torch_model_path, map_location=torch.device('cpu')))
torch_model.eval()
torch_model.to('cpu')

SimpleNet(
  (fc1): Linear(in_features=6, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=32, bias=True)
  (fc5): Linear(in_features=32, out_features=4, bias=True)
  (relu): ReLU()
)

In [30]:
torch.onnx.export(torch_model, torch.rand(size=(1, 6)), 'models/onnx/TA_optimization_NN_cpu.onnx')

In [12]:

dummy_image = torch.rand(size=(1, 6))
pytorch_module = torch_model.eval()

keras_model = nobuco.pytorch_to_keras(
    pytorch_module,
    args=[dummy_image], kwargs=None,
    inputs_channel_order=ChannelOrder.TENSORFLOW,
    outputs_channel_order=ChannelOrder.TENSORFLOW
)

Legend:
    Green — conversion successful
    Yellow — conversion imprecise
    Red — conversion failed
    Red — no converter found
    Bold — conversion applied directly
    * — subgraph reused
    Tensor — this output is not dependent on any of subgraph's input tensors
    Tensor — this input is a parameter / constant
    Tensor — this tensor is useless

SimpleNet[__main__](float32_0<1,6>) -> float32_15<1,4>
 │  Linear[torch.nn.modules.linear](float32_0<1,6>) -> float32_3<1,32>
 │   └· linear[torch.nn.functional](float32_0<1,6>, float32_1<32,6>, float32_2<32>) -> float32_3<1,32>
 │  ReLU[torch.nn.modules.activation](float32_3<1,32>) -> float32_4<1,32>
 │   ├· relu[torch.nn.functional](float32_3<1,32>, inplace=False) -> float32_4<1,32>
 │   └   └· relu[torch](float32_3<1,32>) -> float32_4<1,32>
 │  Linear[torch.nn.modules.linear](float32_4<1,32>) -> float32_7<1,32>
 │   └· linear[torch.nn.functional](float32_4<1,32>, float32_5<32,32>, float32_6<32>) -> float32_7<1,32>
 │  *ReLU[torch

In [13]:
# Test model
import numpy as np
dummy_input = np.random.rand(1, 6)
output = keras_model.predict(dummy_input)
print(output)

1/1 [==============================] - 0s 43ms/step
[[ 0.113675    1.030733    0.47125852 -1.0413678 ]]


In [14]:
# Convert the model to tensorflow format
tf.saved_model.save(keras_model, 'models/tensorflow/TA_optimization_NN_cpu')

INFO:tensorflow:Assets written to: models/tensorflow/TA_optimization_NN_cpu\assets


INFO:tensorflow:Assets written to: models/tensorflow/TA_optimization_NN_cpu\assets


In [15]:
keras_model.save('models/keras/TA_optimization_NN_cpu')
keras_model.sa

INFO:tensorflow:Assets written to: models/keras/TA_optimization_NN_cpu\assets


INFO:tensorflow:Assets written to: models/keras/TA_optimization_NN_cpu\assets


In [25]:
import openvino
# Trace the torch model
dummy_image = torch.rand(size=(1, 6))
torch_model = torch.jit.trace(torch_model, dummy_image).eval()

torch_model.forward(dummy_image)

tensor([[-0.5805,  1.0996,  0.6414, -1.0991]], grad_fn=<AddmmBackward0>)

In [35]:
from openvino import PartialShape

ov_model = openvino.convert_model('models/onnx/TA_optimization_NN_cpu.onnx')
openvino.save_model(ov_model, 'models/openvino/TA_optimization_NN_cpu.xml')